In [1]:
#sample=document=str
doc1='fo@od was Good'
doc2='taste was not good'
doc3='will not re$commend to others'
doc4='service was very prompt'
doc5='awesome Food'
import numpy as np

target=np.array([1,0,0,1,1])

#corpus=list or 1d array of docs
corpus=[doc1,doc2,doc3,doc4,doc5]
corpus2=list(map(str.lower,corpus))
print(corpus2)

import re
def clean_punch(doc):
    doc=re.sub('[^a-z ]','',doc)
    return doc

corpus3=list(map(clean_punch,corpus2))
print(corpus3)

['fo@od was good', 'taste was not good', 'will not re$commend to others', 'service was very prompt', 'awesome food']
['food was good', 'taste was not good', 'will not recommend to others', 'service was very prompt', 'awesome food']


In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [3]:
tk=Tokenizer()
tk.fit_on_texts(corpus3)

In [4]:
tk.index_word

{1: 'was',
 2: 'food',
 3: 'good',
 4: 'not',
 5: 'taste',
 6: 'will',
 7: 'recommend',
 8: 'to',
 9: 'others',
 10: 'service',
 11: 'very',
 12: 'prompt',
 13: 'awesome'}

In [19]:
len(tk.index_word)

13

In [5]:
seqs=tk.texts_to_sequences(corpus3)
print(seqs)

[[2, 1, 3], [5, 1, 4, 3], [6, 4, 7, 8, 9], [10, 1, 11, 12], [13, 2]]


In [6]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [7]:
pad_sequences(seqs,padding='post')

array([[ 2,  1,  3,  0,  0],
       [ 5,  1,  4,  3,  0],
       [ 6,  4,  7,  8,  9],
       [10,  1, 11, 12,  0],
       [13,  2,  0,  0,  0]], dtype=int32)

In [8]:
pad_sequences(seqs,padding='pre')

array([[ 0,  0,  2,  1,  3],
       [ 0,  5,  1,  4,  3],
       [ 6,  4,  7,  8,  9],
       [ 0, 10,  1, 11, 12],
       [ 0,  0,  0, 13,  2]], dtype=int32)

In [9]:
pad_sequences(seqs)

array([[ 0,  0,  2,  1,  3],
       [ 0,  5,  1,  4,  3],
       [ 6,  4,  7,  8,  9],
       [ 0, 10,  1, 11, 12],
       [ 0,  0,  0, 13,  2]], dtype=int32)

In [10]:
pad_sequences(seqs,padding='pre',maxlen=4,truncating='post')

array([[ 0,  2,  1,  3],
       [ 5,  1,  4,  3],
       [ 6,  4,  7,  8],
       [10,  1, 11, 12],
       [ 0,  0, 13,  2]], dtype=int32)

In [11]:
pad_seqs=pad_sequences(seqs,padding='pre',maxlen=4,truncating='post')

In [13]:
from tensorflow.keras.utils import to_categorical

In [16]:
corpus3

['food was good',
 'taste was not good',
 'will not recommend to others',
 'service was very prompt',
 'awesome food']

In [17]:
pad_seqs

array([[ 0,  2,  1,  3],
       [ 5,  1,  4,  3],
       [ 6,  4,  7,  8],
       [10,  1, 11, 12],
       [ 0,  0, 13,  2]], dtype=int32)

In [15]:
X=to_categorical(pad_seqs)
print(X.shape)
X

(5, 4, 14)


array([[[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
    

In [44]:
target

array([1, 0, 0, 1, 1])

In [18]:
from tensorflow.keras.layers import SimpleRNN,Dense,Embedding
from tensorflow.keras.models import Sequential

In [20]:
model=Sequential()
model.add(Embedding(input_dim=len(tk.index_word)+1,output_dim=10))
model.add(SimpleRNN(units=10,activation='tanh'))
model.add(Dense(units=5,activation='relu'))
model.add(Dense(units=1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(pad_seqs,target,batch_size=2,epochs=5)

Epoch 1/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - accuracy: 0.8000 - loss: 0.6899
Epoch 2/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8000 - loss: 0.6795
Epoch 3/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.8000 - loss: 0.6729
Epoch 4/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 1.0000 - loss: 0.6660
Epoch 5/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 1.0000 - loss: 0.6587


In [21]:
import pandas as pd
import numpy as np
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import SimpleRNN,Dense,Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [22]:
df=pd.read_csv('f:/ML_Dataset/food_review_dataset.csv')
df

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1
...,...,...
1995,Everything was on point — from salads to ice c...,1
1996,Highly recommend trying the burger. Simply dis...,1
1997,Poor service and friendly hygiene standards.,0
1998,This is my go-to place for soup. Never disappo...,1


In [23]:
df['length']=list(map(len,df.Review))
df.length.value_counts()

length
55     108
53     107
51      61
60      56
54      55
      ... 
129      1
108      1
145      1
136      1
149      1
Name: count, Length: 134, dtype: int64

In [24]:
corpus=df.Review
y=df.Liked


In [25]:
corpus2=list(map(str.lower,corpus))

In [26]:
def clean_punch(doc):
    doc=re.sub('[^a-z ]','',doc)
    return doc

corpus3=list(map(clean_punch,corpus2))
tk=Tokenizer()
tk.fit_on_texts(corpus3)
print(len(tk.index_word))
seqs=tk.texts_to_sequences(corpus3)
pad_seqs=pad_sequences(seqs,padding='pre',maxlen=70,truncating='post')

2064


In [29]:
model=Sequential()
model.add(Embedding(input_dim=len(tk.index_word)+1,output_dim=200))
model.add(SimpleRNN(units=10,activation='tanh'))
model.add(Dense(units=10,activation='relu'))
model.add(Dense(units=1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(pad_seqs,y,batch_size=32,epochs=10,validation_split=.2)

Epoch 1/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - accuracy: 0.7181 - loss: 0.5646 - val_accuracy: 1.0000 - val_loss: 0.1736
Epoch 2/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9325 - loss: 0.2899 - val_accuracy: 1.0000 - val_loss: 0.0795
Epoch 3/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9794 - loss: 0.1413 - val_accuracy: 1.0000 - val_loss: 0.0364
Epoch 4/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.9925 - loss: 0.0696 - val_accuracy: 1.0000 - val_loss: 0.0205
Epoch 5/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.9962 - loss: 0.0389 - val_accuracy: 1.0000 - val_loss: 0.0118
Epoch 6/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9987 - loss: 0.0221 - val_accuracy: 1.0000 - val_loss: 0.0082
Epoch 7/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9994 - loss: 0.0141 - val_accuracy: 1.0000 - val_loss: 0.0065
Epoch 8/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 1.0000 - loss: 0.0098 - val_accuracy: 1.0000 - v

In [30]:
new_sample1='food was good'
new_sample2='we did not like the food'

test_corpus=[new_sample1,new_sample2]
test_corpus2=list(map(str.lower,test_corpus))
test_corpus3=list(map(clean_punch,test_corpus2))

seqs=tk.texts_to_sequences(test_corpus3)
pad_seqs=pad_sequences(seqs,padding='pre',maxlen=70,truncating='post')
model.predict(pad_seqs)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step


array([[0.8316051 ],
       [0.04384704]], dtype=float32)

In [72]:
df=pd.read_csv('f:/ML_Dataset/language_detection_dataset.csv')
df

,text,label
0,وہ بازار جا رہی ہے۔,URDU
1,यह एक सुंदर दिन है।,HINDI
2,She is going to the market.,ENGLISH
3,ਅੱਜ ਮੌਸਮ ਬਹੁਤ ਵਧੀਆ ਹੈ।,PUNJABI
4,मुझे किताबें पढ़ना पसंद है।,HINDI
...,...,...
4995,مجھے کتابیں پڑھنا پسند ہے۔,URDU
4996,ਇਹ ਇੱਕ ਸੋਹਣਾ ਦਿਨ ਹੈ।,PUNJABI
4997,भारतः महत्तमः देशः अस्ति।,SANSKRIT
4998,अद्य मौसमः अतीव उत्तमः अस्ति।,SANSKRIT


In [73]:
df.label.value_counts()
df.duplicated().sum()

np.int64(4975)

In [74]:
#df.drop_duplicates(inplace=True)

In [75]:
from sklearn.preprocessing import LabelEncoder

In [76]:
enc=LabelEncoder()
df.label=enc.fit_transform(df.label)

In [77]:
df

,text,label
0,وہ بازار جا رہی ہے۔,4
1,यह एक सुंदर दिन है।,1
2,She is going to the market.,0
3,ਅੱਜ ਮੌਸਮ ਬਹੁਤ ਵਧੀਆ ਹੈ।,2
4,मुझे किताबें पढ़ना पसंद है।,1
...,...,...
4995,مجھے کتابیں پڑھنا پسند ہے۔,4
4996,ਇਹ ਇੱਕ ਸੋਹਣਾ ਦਿਨ ਹੈ।,2
4997,भारतः महत्तमः देशः अस्ति।,3
4998,अद्य मौसमः अतीव उत्तमः अस्ति।,3


In [78]:
df['length']=list(map(len,df.text))
df.length.value_counts()

length
20    813
25    632
22    601
19    596
27    557
31    425
21    414
26    209
23    201
17    186
29    185
35    181
Name: count, dtype: int64

In [79]:
corpus=df.text
y=df.label

In [82]:
tk=Tokenizer()
tk.fit_on_texts(corpus)
print(len(tk.index_word))
seqs=tk.texts_to_sequences(corpus)
pad_seqs=pad_sequences(seqs,padding='pre',maxlen=20,truncating='post')

98


In [83]:
import tensorflow as tf
import numpy as np
import random

In [85]:
tf.random.set_seed(10)
np.random.seed(10)
random.seed(10)
model=Sequential()
model.add(Embedding(input_dim=len(tk.index_word)+1,output_dim=70))
model.add(SimpleRNN(units=3,activation='tanh'))
model.add(Dense(units=5,activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(pad_seqs,y,batch_size=10,epochs=2,validation_split=.2)

Epoch 1/2
400/400 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9890 - loss: 0.6959 - val_accuracy: 1.0000 - val_loss: 0.4318
Epoch 2/2
400/400 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 1.0000 - loss: 0.3178 - val_accuracy: 1.0000 - val_loss: 0.2440


In [97]:
new_sample1='sky is the limit'
new_sample2='खाना अच्छा था'

test_corpus=[new_sample1,new_sample2]

seqs=tk.texts_to_sequences(test_corpus)
pad_seqs=pad_sequences(seqs,padding='pre',maxlen=20,truncating='post')
np.argmax(model.predict(pad_seqs),axis=1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step


array([3, 0])

In [98]:
enc.inverse_transform(res)

array(['SANSKRIT', 'SANSKRIT'], dtype=object)